# Data Preparation

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
!pip install gdown
!gdown --id 1my5Xr0NSYaEsgwaDEG9HkaK1LOlNsnFJ

# Unzip the file
!unzip -q dataset2.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1my5Xr0NSYaEsgwaDEG9HkaK1LOlNsnFJ
From (redirected): https://drive.google.com/uc?id=1my5Xr0NSYaEsgwaDEG9HkaK1LOlNsnFJ&confirm=t&uuid=4accca37-8d03-4de6-bf57-3b5f406d5fe8
To: /content/dataset2.zip
100% 810M/810M [00:26<00:00, 30.1MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
TRAINING_DIR = '/content/dataset/TRAIN'
VALIDATION_DIR = '/content/dataset/TEST'

training_datagen = ImageDataGenerator(
    rescale=1./255
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255
)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size = (150,150),
    class_mode = 'categorical',
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size = (150,150),
    class_mode = 'categorical',
)

Found 1490 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


# Model

## CNN Model

In [ ]:
import tensorflow as tf

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])


##Train Model

In [ ]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    verbose=1
)

Epoch 1/100
47/47 [==============================] - 34s 527ms/step - loss: 2.3009 - accuracy: 0.0987 - val_loss: 2.2912 - val_accuracy: 0.1400
Epoch 2/100
47/47 [==============================] - 24s 506ms/step - loss: 2.2530 - accuracy: 0.1658 - val_loss: 2.1867 - val_accuracy: 0.1950
Epoch 3/100
47/47 [==============================] - 23s 500ms/step - loss: 2.1452 - accuracy: 0.2060 - val_loss: 2.1193 - val_accuracy: 0.2350
Epoch 4/100
47/47 [==============================] - 23s 482ms/step - loss: 2.0954 - accuracy: 0.2282 - val_loss: 2.0479 - val_accuracy: 0.2900
Epoch 5/100
47/47 [==============================] - 23s 500ms/step - loss: 2.0393 - accuracy: 0.2591 - val_loss: 2.2146 - val_accuracy: 0.1950
Epoch 6/100
47/47 [==============================] - 24s 500ms/step - loss: 1.9927 - accuracy: 0.2664 - val_loss: 1.9695 - val_accuracy: 0.3550
Epoch 7/100
47/47 [==============================] - 23s 484ms/step - loss: 1.9679 - accuracy: 0.2993 - val_loss: 1.8931 - val_accuracy:

## TEST MODEL


In [ ]:
motivs = train_generator.class_indices
print(motivs)

{'Batik Betawi': 0, 'Batik Blekok': 1, 'Batik Cendrawasih': 2, 'Batik Geblek Renteng': 3, 'Batik Kawung': 4, 'Batik Lasem': 5, 'Batik Megamendung': 6, 'Batik Parang': 7, 'Batik Poleng': 8, 'Batik Sekar Jagad': 9}


In [ ]:
test_dir = '/content/dataset/TEST'

for motive in motivs.keys():
    images_list = os.listdir(os.path.join(test_dir, motive))
    # plt.figure(figsize=(16, 20))

    correct_count = 0

    for idx, image in enumerate(images_list):
        img_path = f'/content/dataset/TEST/{motive}/{image}'
        img = load_img(img_path, target_size=(150, 150))

        img_array = img_to_array(img) / 255.0
        img_array = tf.expand_dims(img_array, 0)

        motives_list = list(motivs.keys())
        prediction = model(img_array)
        pred_idx = np.argmax(prediction)
        pred_motive = motives_list[pred_idx]
        pred_confidence = prediction[0][pred_idx] * 100

        if pred_motive == motive:
            correct_count += 1

        # plt.subplot(5, 4, idx+1)
        # plt.title(f'Actual: {motive}\nPrediction: {pred_motive} {pred_confidence:.2f}%')
        # plt.imshow(img)
        # plt.axis('off')

    print(f'{motive} Predictions --> {correct_count}/20 correct')
# plt.show()

Batik Betawi Predictions --> 14/20 correct
Batik Blekok Predictions --> 20/20 correct
Batik Cendrawasih Predictions --> 13/20 correct
Batik Geblek Renteng Predictions --> 17/20 correct
Batik Kawung Predictions --> 16/20 correct
Batik Lasem Predictions --> 11/20 correct
Batik Megamendung Predictions --> 16/20 correct
Batik Parang Predictions --> 10/20 correct
Batik Poleng Predictions --> 10/20 correct
Batik Sekar Jagad Predictions --> 17/20 correct


# SAVE MODEL h.5


In [ ]:
import tensorflow as tf
from google.colab import files

model.save('modelCNN.h5')